In [0]:
%pip install axios python-jose

  Obtaining dependency information for axios from https://files.pythonhosted.org/packages/f9/8b/26c598e8971ee75e14479db4815391e54517a2245a7b4a644107984e9ce0/axios-0.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for python-jose from https://files.pythonhosted.org/packages/bd/2d/e94b2f7bab6773c70efc70a61d66e312e1febccd9e0db6b9e0adf58cbad1/python_jose-3.3.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/42/07/b29571a58a3a80681722ea8ed0ba569211d9bb8531ad49b5cacf6d409185/lxml-5.3.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for rich from https://files.pythonhosted.org/packages/b0/11/dadb85e2bd6b1f1ae56669c3e1f0410797f9605d752d68fb47b77f525b31/rich-13.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for ecdsa!=0.15 from https://files.pythonhosted.org/packages/00/e7/ed3243b30d1bec41675b6394a1daae46349dc2b855cb83be846a5a918238/ecdsa-0.19.0-py2.py3-no

In [0]:
dbutils.library.restartPython()

In [0]:
# TODO: move key file to key vault
with open('/Volumes/yamada_dev/redox_fhir/config/private key.pem', 'rb') as f:
  private_key = f.read()

## TODO: 1. read client id from config and/or secrets; 2. read kid from key file and/or secrets 
redox_client_id = '40c57cc3-e4b5-4bb5-a19d-a7cc77204fa5'
private_key_kid = 'JKMTAeKln8EIpbuNePD7Lm0HM_yUkojryb1tnm2Z7Gc'
redox_auth_location = 'https://api.redoxengine.com/v2/auth/token'
redox_base_url = 'https://api.redoxengine.com/fhir/R4/redox-fhir-sandbox/Development/'
## All Redox FHIR request URLs start with this base: https://api.redoxengine.com/fhir/R4/[organization-name]/[environment-type]/

In [0]:
observation_bundle = """{
  "resourceType": "Bundle",
  "type": "message",
  "entry": [
    {
      "resource": {
        "category": [
          {
            "coding": [
              {
                "code": "vital-signs",
                "display": "vital-signs",
                "system": "http://terminology.hl7.org/CodeSystem/observation-category"
              }
            ]
          }
        ],
        "code": {
          "coding": [
            {
              "code": "8302-2",
              "display": "Body Height",
              "system": "http://loinc.org"
            }
          ],
          "text": "Body Height"
        },
        "effectiveDateTime": "2020-12-15T18:06:33-05:00",
        "id": "acbeab78-648d-1ab1-37b9-77d91c528950",
        "issued": "2020-12-15T18:06:33.245-05:00",
        "meta": {
          "lastUpdated": "2024-07-15T15:38:09.805744+00:00",
          "profile": [
            "http://hl7.org/fhir/StructureDefinition/bodyheight",
            "http://hl7.org/fhir/StructureDefinition/vitalsigns"
          ],
          "versionId": "MTcyMTA1Nzg4OTgwNTc0NDAwMA"
        },
        "resourceType": "Observation",
        "status": "final",
        "subject": {
          "reference": "Patient/58117110-ae47-452a-be2c-2d82b3a9e24b"
        },
        "valueQuantity": {
          "code": "cm",
          "system": "http://unitsofmeasure.org",
          "unit": "cm",
          "value": 404.2
        }
      }
    }
  ]
}"""

In [0]:
import datetime
import jwt
import requests
from uuid import uuid4

class BackendServiceAuth(requests.auth.AuthBase):

  def __init__(self, auth_location, client_id, kid, private_key):
    self.auth_location = auth_location
    self.client_id = client_id
    self.kid = kid
    self.private_key = private_key
    self.token = None

  def __call__(self, request):
    if not self.token:
      expiration = datetime.datetime.now() + datetime.timedelta(minutes=5)

      headers = {
        'kid': self.kid,
        'alg': 'RS384',
        'typ': 'JWT',
      }

      assertion = jwt.encode(
        {
          'iss': self.client_id,
          'sub': self.client_id,
          'aud': self.auth_location,
          'exp': int(expiration.strftime('%s')),
          'iat': int(datetime.datetime.now().strftime('%s')),
          'jti': uuid4().hex,
        },
        self.private_key,
        algorithm='RS384',
        headers=headers)

      payload = {
        'grant_type': 'client_credentials',
        'client_assertion_type':
        'urn:ietf:params:oauth:client-assertion-type:jwt-bearer',
        'client_assertion': assertion
      }
      try:
        response = requests.post(self.auth_location, data=payload, timeout=30)
        response.raise_for_status()
        self.token = response.json()['access_token']
        print(f"{datetime.datetime.now()} - Authentication successful!")
      except requests.exceptions.RequestException as e:
        if hasattr(e, 'response') and e.response is not None:
          print(f"{datetime.datetime.now()} - Request failed with status code: {e.response.status_code}")
          print(f"{datetime.datetime.now()} - Response body: {e.response.text}")
        else:
          print(f"{datetime.datetime.now()} - Request failed: {e}")

    request.headers['Authorization'] = 'Bearer %s' % self.token
    return request

In [0]:
def redox_fhir_api_action(client_id, kid, auth_location, private_key, base_url, http_method, resource, interaction, bundle=None):
    auth = BackendServiceAuth(auth_location, client_id, kid, private_key)

    method = getattr(requests, http_method)

    endpoint = resource+'/'+interaction

    response = method(base_url+endpoint, auth=auth, data=bundle)
    
    if response.status_code == 200:
        try:
            print(f"{datetime.datetime.now()} - {response.json()}")
        except JSONDecodeError:
            print(f"{datetime.datetime.now()} - Failed to decode JSON from response.")
    else:
        print(f"{datetime.datetime.now()} - Request failed with status code: {response.status_code}")
        print(f"{datetime.datetime.now()} - Error message from endpoint: {response.text}")

In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Patient','_search')

2024-09-20 17:37:29.684775 - Authentication successful!
2024-09-20 17:37:30.471389 - {'entry': [{'fullUrl': 'https://fhir.redoxengine.com/fhir-sandbox/Patient/ccfc4db2-2026-7adb-3db0-33f3828140bb', 'resource': {'address': [{'city': 'Peabody', 'country': 'US', 'extension': [{'extension': [{'url': 'latitude', 'valueDecimal': 42.53488867467872}, {'url': 'longitude', 'valueDecimal': -70.9597319140293}], 'url': 'http://hl7.org/fhir/StructureDefinition/geolocation'}], 'line': ['768 Mertz Camp'], 'postalCode': '01940', 'state': 'MA'}], 'birthDate': '1976-06-06', 'communication': [{'language': {'coding': [{'code': 'en-US', 'display': 'English', 'system': 'urn:ietf:bcp:47'}], 'text': 'English'}}], 'extension': [{'extension': [{'url': 'ombCategory', 'valueCoding': {'code': '2106-3', 'display': 'White', 'system': 'urn:oid:2.16.840.1.113883.6.238'}}, {'url': 'text', 'valueString': 'White'}], 'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'}, {'extension': [{'url': 'ombCategory

In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'get', 'Patient','58117110-ae47-452a-be2c-2d82b3a9e24b')

2024-09-20 17:37:45.723558 - Authentication successful!
2024-09-20 17:37:46.269366 - {'address': [{'city': 'Monroe', 'country': 'US', 'district': 'Greene', 'line': ['4762 Hickory Street'], 'postalCode': '53566', 'state': 'WI', 'use': 'home'}], 'birthDate': '1963-02-02', 'communication': [{'language': {'coding': [{'code': 'en', 'system': 'urn:ietf:bcp:47'}], 'text': 'English'}}], 'contact': [{'address': {'city': 'Monroe', 'country': 'USA', 'district': 'Green', 'line': ['4762 Hickory St.'], 'postalCode': '53566', 'state': 'WI'}, 'name': {'family': 'Bixby', 'given': ['Barbara']}, 'relationship': [{'coding': [{'code': 'MTH', 'display': 'mother', 'system': 'http://terminology.hl7.org/CodeSystem/v3-RoleCode'}], 'text': 'Mother'}, {'coding': [{'code': 'C', 'display': 'Emergency Contact', 'system': 'http://terminology.hl7.org/CodeSystem/v2-0131'}], 'text': 'Emergency Contact'}], 'telecom': [{'system': 'phone', 'use': 'home', 'value': '+18088675303'}, {'system': 'email', 'value': 'barbara.bixby

In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Observation','$observation-create', observation_bundle)

2024-09-20 17:38:13.708558 - Authentication successful!
2024-09-20 17:38:14.307075 - {'entry': [{'response': {'etag': 'W/"MTcyNjg1Mzg5NDIyODkwODAwMA"', 'lastModified': '2024-09-20T17:38:14.228908+00:00', 'location': 'https://fhir.redoxengine.com/fhir-sandbox/Observation/2ddc5862-27f4-4663-90c1-0773c337ea48/_history/MTcyNjg1Mzg5NDIyODkwODAwMA', 'status': '201 Created'}}], 'resourceType': 'Bundle', 'type': 'transaction-response'}


In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'get', 'Observation','2ddc5862-27f4-4663-90c1-0773c337ea48')

2024-09-20 17:44:42.506150 - Authentication successful!
2024-09-20 17:44:42.982087 - {'category': [{'coding': [{'code': 'vital-signs', 'display': 'vital-signs', 'system': 'http://terminology.hl7.org/CodeSystem/observation-category'}]}], 'code': {'coding': [{'code': '8302-2', 'display': 'Body Height', 'system': 'http://loinc.org'}], 'text': 'Body Height'}, 'effectiveDateTime': '2020-12-15T18:06:33-05:00', 'id': '2ddc5862-27f4-4663-90c1-0773c337ea48', 'issued': '2020-12-15T18:06:33.245-05:00', 'meta': {'lastUpdated': '2024-09-20T17:38:14.228908+00:00', 'profile': ['http://hl7.org/fhir/StructureDefinition/bodyheight', 'http://hl7.org/fhir/StructureDefinition/vitalsigns'], 'versionId': 'MTcyNjg1Mzg5NDIyODkwODAwMA'}, 'resourceType': 'Observation', 'status': 'final', 'subject': {'reference': 'Patient/58117110-ae47-452a-be2c-2d82b3a9e24b'}, 'valueQuantity': {'code': 'cm', 'system': 'http://unitsofmeasure.org', 'unit': 'cm', 'value': 404.2}}


In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Appointment','$appointment-create', observation_bundle)

2024-09-20 13:21:46.994412 - Authentication successful!
2024-09-20 13:21:47.601996 - {'entry': [{'response': {'etag': 'W/"MTcyNjgzODUwNzQwMDAyMDAwMA"', 'lastModified': '2024-09-20T13:21:47.400020+00:00', 'location': 'https://fhir.redoxengine.com/fhir-sandbox/Observation/f47516c0-adf5-40ff-8ae0-2e79a60df5ec/_history/MTcyNjgzODUwNzQwMDAyMDAwMA', 'status': '201 Created'}}], 'resourceType': 'Bundle', 'type': 'transaction-response'}


In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Claim','$submit-preauthorization', observation_bundle)

2024-09-20 13:47:41.008137 - Authentication successful!
2024-09-20 13:47:41.653674 - {'entry': [{'response': {'etag': 'W/"MTcyNjg0MDA2MTU0OTQyODAwMA"', 'lastModified': '2024-09-20T13:47:41.549428+00:00', 'location': 'https://fhir.redoxengine.com/fhir-sandbox/Observation/bf23aa03-9e00-4281-b96c-a2c365bc52c7/_history/MTcyNjg0MDA2MTU0OTQyODAwMA', 'status': '201 Created'}}], 'resourceType': 'Bundle', 'type': 'transaction-response'}


In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Observation','$appointment-create', observation_bundle)

2024-09-20 14:24:18.138396 - Authentication successful!
2024-09-20 14:24:18.295569 - Request failed with status code: 400
2024-09-20 14:24:18.295618 - Error message from endpoint: {"Meta":{"Logs":[{"ID":"b4265230-17ae-4af0-9356-a5ec5092ed25","AttemptID":"97b2411a-2eac-4b94-a281-3840c3ace276"}],"Source":{"ID":"91b0ab2f-7b86-441d-9cbf-9b9a6d648d59","Name":"Redox Request Ingress (Development)"},"Errors":[{"Text":"Invalid data model and/or event type.","Type":"message","Module":"Parser"}]}}


In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Fake','$fake-create', observation_bundle)

2024-09-20 14:23:10.648955 - Authentication successful!
2024-09-20 14:23:10.940295 - Request failed with status code: 404
2024-09-20 14:23:10.940404 - Error message from endpoint: {"resourceType":"OperationOutcome","issue":[{"severity":"fatal","code":"not-found","details":{"coding":[{"code":"MSG_NO_MODULE","system":"http://terminology.hl7.org/CodeSystem/operation-outcome"}],"text":"Unknown slug redox-fhir-sandbox"}}]}


In [0]:
appointment_bundle = """{
  "resourceType": "Bundle",
  "type": "message",
  "entry": [
    {
      "resource": {
        "comment": "This appointment is scheduled for an encounter related to symptoms and reports of violence in the environment.",
        "created": "2021-02-23T08:00:00Z",
        "description": "Appointment for an encounter related to symptoms and reports of violence in the environment",
        "end": "2021-02-24T09:45:03-05:00",
        "id": "ca4006b5-67e7-43c8-ba4e-b5891b6245c4",
        "identifier": [
          {
            "system": "https://github.com/synthetichealth/synthea",
            "use": "official",
            "value": "5b468738-97d5-6172-2060-a9fdda303919"
          }
        ],
        "meta": {
          "lastUpdated": "2024-07-15T15:38:08.667364+00:00",
          "profile": [
            "http://hl7.org/fhir/us/core/StructureDefinition/us-core-appointment"
          ],
          "versionId": "MTcyMTA1Nzg4ODY2NzM2NDAwMA"
        },
        "participant": [
          {
            "actor": {
              "reference": "Patient/58117110-ae47-452a-be2c-2d82b3a9e24b"
            },
            "required": "required",
            "status": "accepted"
          },
          {
            "actor": {
              "display": "Dr. Lesley Fisher",
              "reference": "Practitioner/3aade15e-5333-3c4e-86c5-aab3511bc8e8"
            },
            "required": "required",
            "status": "accepted"
          },
          {
            "actor": {
              "display": "MOUNT AUBURN HOSPITAL",
              "reference": "Location/27abf3f3-fa08-31e4-8bca-c4c9ade68709"
            },
            "required": "required",
            "status": "accepted"
          }
        ],
        "reasonCode": [
          {
            "coding": [
              {
                "code": "424393004",
                "display": "Reports of violence in the environment (finding)",
                "system": "http://snomed.info/sct"
              }
            ],
            "text": "Reports of violence in the environment (finding)"
          }
        ],
        "resourceType": "Appointment",
        "serviceCategory": [
          {
            "coding": [
              {
                "code": "gp",
                "display": "General Practice",
                "system": "http://terminology.hl7.org/CodeSystem/service-category"
              }
            ]
          }
        ],
        "serviceType": [
          {
            "coding": [
              {
                "code": "185345009",
                "display": "Encounter for symptom (procedure)",
                "system": "http://snomed.info/sct"
              }
            ],
            "text": "Encounter for symptom (procedure)"
          }
        ],
        "start": "2021-02-24T08:12:16-05:00",
        "status": "booked"
      }
    }
  ]
}"""

In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Appointment','$appointment-create', appointment_bundle)

2024-09-20 13:28:08.834086 - Authentication successful!
2024-09-20 13:28:09.614475 - {'entry': [{'response': {'etag': 'W/"MTcyNjgzODg4OTQ0MTUzNDAwMA"', 'lastModified': '2024-09-20T13:28:09.441534+00:00', 'location': 'https://fhir.redoxengine.com/fhir-sandbox/Appointment/ca4006b5-67e7-43c8-ba4e-b5891b6245c4/_history/MTcyNjgzODg4OTQ0MTUzNDAwMA', 'status': '200 OK'}}], 'resourceType': 'Bundle', 'type': 'transaction-response'}


In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'get', 'Appointment','ca4006b5-67e7-43c8-ba4e-b5891b6245c4')

2024-09-20 13:29:48.950947 - Authentication successful!
2024-09-20 13:29:49.480791 - {'comment': 'This appointment is scheduled for an encounter related to symptoms and reports of violence in the environment.', 'created': '2021-02-23T08:00:00Z', 'description': 'Appointment for an encounter related to symptoms and reports of violence in the environment', 'end': '2021-02-24T09:45:03-05:00', 'id': 'ca4006b5-67e7-43c8-ba4e-b5891b6245c4', 'identifier': [{'system': 'https://github.com/synthetichealth/synthea', 'use': 'official', 'value': '5b468738-97d5-6172-2060-a9fdda303919'}], 'meta': {'lastUpdated': '2024-09-20T13:28:09.441534+00:00', 'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-appointment'], 'versionId': 'MTcyNjgzODg4OTQ0MTUzNDAwMA'}, 'participant': [{'actor': {'reference': 'Patient/58117110-ae47-452a-be2c-2d82b3a9e24b'}, 'required': 'required', 'status': 'accepted'}, {'actor': {'display': 'Dr. Lesley Fisher', 'reference': 'Practitioner/3aade15e-5333-3c4e-86c5-aa

In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'get', 'Appointment','ca4006b5-67e7-43c8-ba4e-b5891b6245c4', appointment_bundle)

2024-09-20 14:18:43.645485 - Authentication successful!
2024-09-20 14:18:44.319374 - {'comment': 'This appointment is scheduled for an encounter related to symptoms and reports of violence in the environment.', 'created': '2021-02-23T08:00:00Z', 'description': 'Appointment for an encounter related to symptoms and reports of violence in the environment', 'end': '2021-02-24T09:45:03-05:00', 'id': 'ca4006b5-67e7-43c8-ba4e-b5891b6245c4', 'identifier': [{'system': 'https://github.com/synthetichealth/synthea', 'use': 'official', 'value': '5b468738-97d5-6172-2060-a9fdda303919'}], 'meta': {'lastUpdated': '2024-09-20T13:28:09.441534+00:00', 'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-appointment'], 'versionId': 'MTcyNjgzODg4OTQ0MTUzNDAwMA'}, 'participant': [{'actor': {'reference': 'Patient/58117110-ae47-452a-be2c-2d82b3a9e24b'}, 'required': 'required', 'status': 'accepted'}, {'actor': {'display': 'Dr. Lesley Fisher', 'reference': 'Practitioner/3aade15e-5333-3c4e-86c5-aa

In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Appointment','ca4006b5-67e7-43c8-ba4e-b5891b6245c4', appointment_bundle)

2024-09-20 14:19:48.598428 - Authentication successful!
2024-09-20 14:19:48.751858 - Request failed with status code: 404
2024-09-20 14:19:48.751904 - Error message from endpoint: 


In [0]:
redox_fhir_api_action(redox_client_id, private_key_kid, redox_auth_location, private_key, redox_base_url, 'post', 'Observation','$observation-create', appointment_bundle)

2024-09-20 15:04:20.963379 - Authentication successful!
2024-09-20 15:04:21.786483 - {'entry': [{'response': {'etag': 'W/"MTcyNjg0NDY2MTQ4MzI5MjAwMA"', 'lastModified': '2024-09-20T15:04:21.483292+00:00', 'location': 'https://fhir.redoxengine.com/fhir-sandbox/Appointment/ca4006b5-67e7-43c8-ba4e-b5891b6245c4/_history/MTcyNjg0NDY2MTQ4MzI5MjAwMA', 'status': '200 OK'}}], 'resourceType': 'Bundle', 'type': 'transaction-response'}
